In [4]:
# !pip install ibm-watson==7.* pandas
!pip install ibm-watson pandas

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
Using cached PyJWT-2.10.1-py3-none-any.whl (22 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
  Created wheel for ibm-watson: filename=ibm_watson-11.0.0-py3-none-any.whl size=361695 sha256=d0e4bb5b2698f2d0ace3fd63f3304115ce061dc7c1a1f991824fc9ddcf2925b8
  Stored in directory: /Users/ivanp/Library/Caches/pip/wheels/d0/17/97/8398dbd81b9628280393ba5d78aff126302313435f7325f302
Successfully built ibm-watson
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [ibm-watson]9 [ibm-watson]


In [6]:
!pip install python-dotenv

In [1]:
import os
from collections import Counter, defaultdict
import pandas as pd
from dotenv import load_dotenv
from ibm_watson import AssistantV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator


# --- CONFIG ---


# Load environment variables from .env if present
load_dotenv()

# Read from environment (with defaults to empty string)
API_KEY = os.getenv("WA_API_KEY", "")
SERVICE_URL = os.getenv("WA_URL", "https://api.us-south.assistant.watson.cloud.ibm.com")
WORKSPACE_ID = os.getenv("WA_WORKSPACE_ID", "")

# --- CONNECT ---
authenticator = IAMAuthenticator(API_KEY)
assistant = AssistantV1(
    version="2021-06-14",
    authenticator=authenticator
)
assistant.set_service_url(SERVICE_URL)

print("Watson Assistant client initialized.")


Watson Assistant client initialized.


In [2]:
# --- TEST MESSAGE ---
response = assistant.message(
    workspace_id=WORKSPACE_ID,
    input={"text": "What flowers do you recommend for Valentine's Day?"}
).get_result()

print(response["intents"])
print(response["output"]["text"])

[{'intent': 'flower_recommendations', 'confidence': 0.977379322052002}]
['Thank you, .', 'You can never go wrong with a dozen red roses.']


In [10]:
# 🧩 SECTION: Evaluate Watson Assistant intents (Student Exercise)

# 🔁 Alias map (your old names → workspace names shown in your screenshot)
INTENT_ALIASES = {
    "recommend_flowers": "flower_recommendations",
    "hours": "hours_info",
    "store_location": "location_info",
    # keep if you truly have it in your workspace:
    "availability": "availability",  # remove if not present
}

# 📚 Test set
TEST_ITEMS = [
    ("Which flowers for Valentine's Day?", "recommend_flowers"),
    ("Are you open on Sundays?", "hours"),
    ("Where are your stores?", "store_location"),
    ("Do you have red roses in stock?", "availability"),
]

# --- CONNECT ---
# TODO: ❶ Initialize the Watson Assistant client here using API_KEY, SERVICE_URL, WORKSPACE_ID
# authenticator = ...
# assistant = ...
# assistant.set_service_url(...)

# --- DISCOVER INTENTS ---
# TODO: ❷ Retrieve intents from your workspace and print them
# workspace_intents = assistant.list_intents(...).get_result()
# actual_intent_names = ...
# print(actual_intent_names)

# --- EVALUATION LOOP ---
rows = []

# Define helper function for alias matching
def is_alias_match(expected, predicted):
    """Return True if predicted equals expected OR equals alias(expected)."""
    return predicted == expected or predicted == INTENT_ALIASES.get(expected, None)

for text, expected in TEST_ITEMS:
    # TODO: ❸ Send message to the Assistant API and capture top intent and confidence
    # resp = assistant.message(...).get_result()
    # top_intent = ...
    # conf = ...
    
    # --- student complete here ---
    strict_ok = ___   # True if top_intent == expected
    alias_ok  = ___   # True if is_alias_match(expected, top_intent)
    
    rows.append({
        "utterance": text,
        "expected_intent": expected,
        "predicted_intent": top_intent,
        "confidence": round(conf, 3),
        "strict_correct": strict_ok,
        "alias_correct": alias_ok
    })

df = pd.DataFrame(rows)

# --- METRICS (students complete this section) ---
N = len(df)

# TODO: ❹ Compute strict accuracy and alias-adjusted accuracy
# strict_acc = ...
# alias_acc  = ...

print("\n=== Watson Assistant Evaluation ===")
print(f"Samples: {N}")
print(f"Strict Accuracy:        {strict_acc:.2%}")
print(f"Alias-Adjusted Accuracy:{alias_acc:.2%}\n")

# TODO: ❺ Compute a simple confusion matrix (optional bonus)
# Hint: use pd.crosstab(df["expected_intent"], df["predicted_intent"])
# cm = ...
# print(cm)

# --- Reflection (students discuss in class) ---
# ✅ What does the difference between strict and alias accuracy tell you?
# ✅ When would alias-adjusted accuracy be a better indicator?
# ✅ How could you extend this evaluation to include precision/recall or F1?


=== Watson Assistant Evaluation ===
Samples: 4
Strict Accuracy:        0.00%
Alias-Adjusted Accuracy:86.67%



In [12]:


# ✅ SOLUTION (with comments): Evaluate Watson Assistant intents

# ---- Imports & setup ---------------------------------------------------------
# pandas: tabular handling; collections: lightweight counters/CMs
import os
import pandas as pd
from collections import defaultdict, Counter

# dotenv: lets you keep secrets in a local .env during development
from dotenv import load_dotenv

# IBM Watson Assistant (Classic) SDK
from ibm_watson import AssistantV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# ---- Load configuration from environment ------------------------------------
# This reads a local .env file if present (safe for local dev / student laptops).
load_dotenv()

API_KEY = os.getenv("WA_API_KEY", "")
SERVICE_URL = os.getenv("WA_URL", "https://api.us-south.assistant.watson.cloud.ibm.com")
WORKSPACE_ID = os.getenv("WA_WORKSPACE_ID", "")

# Fast fail with a clear error if something is missing
assert API_KEY, "Missing WA_API_KEY (set in .env or export in your shell)"
assert SERVICE_URL, "Missing WA_URL (set in .env or export in your shell)"
assert WORKSPACE_ID, "Missing WA_WORKSPACE_ID (set in .env or export in your shell)"

# ---- Alias map & test set ----------------------------------------------------
# INTENT_ALIASES lets us tolerate renamed intents (e.g., workspace uses 'flower_recommendations'
# while our exercise still expects 'recommend_flowers'). This prevents false negatives.
INTENT_ALIASES = {
    "recommend_flowers": "flower_recommendations",
    "hours": "hours_info",
    "store_location": "location_info",
    # keep only if it truly exists in your workspace:
    "availability": "availability",
}

# A compact test set (you can extend it). Each item: (utterance, expected_intent_label)
TEST_ITEMS = [
    ("Which flowers for Valentine's Day?", "recommend_flowers"),
    ("What flowers do you suggest?", "recommend_flowers"),
    ("Flowers suggestions for my girlfriend", "recommend_flowers"),
    ("Can you recommend any flowers?", "recommend_flowers"),
    ("Can you make a recommendation for a Birthday?", "recommend_flowers"),

    ("When do you open", "hours"),
    ("What are your hours of operation?", "hours"),
    ("Are you open on Saturdays?", "hours"),
    ("Are you open on Sundays?", "hours"),

    ("Where are your stores?", "store_location"),
    ("Where are you physically located?", "store_location"),
    ("what's the address of your Vancouver store?", "store_location"),
    ("what's the address of your Toronto store?", "store_location"),

    ("Do you have red roses in stock?", "availability"),
    ("Are peonies available this week?", "availability"),
]

# ---- Connect to Assistant ----------------------------------------------------
# Classic IAM auth + service URL. Version is the API date.
authenticator = IAMAuthenticator(API_KEY)
assistant = AssistantV1(version="2021-06-14", authenticator=authenticator)
assistant.set_service_url(SERVICE_URL)

# ---- Optional: Inspect workspace intents ------------------------------------
# Useful for sanity checks and showing students how to verify labels exist.
workspace_intents = assistant.list_intents(
    workspace_id=WORKSPACE_ID, export=False
).get_result()
actual_intent_names = sorted([i["intent"] for i in workspace_intents.get("intents", [])])

print("\n=== Intents in workspace ===")
print(actual_intent_names)

# Warn students if any expected (after alias mapping) are absent in the workspace.
unknown_expected = set()
for _, expected in TEST_ITEMS:
    target = INTENT_ALIASES.get(expected, expected)
    if target not in actual_intent_names:
        unknown_expected.add((expected, target))
if unknown_expected:
    print("\n⚠️  Warning: Some expected labels (after alias mapping) are not in this workspace:")
    for old, mapped in sorted(unknown_expected):
        print(f"  - expected '{old}' → mapped '{mapped}'  (NOT FOUND)")

# ---- Evaluation loop ---------------------------------------------------------
# We'll collect per-utterance rows and two confusion trackers:
#  - 'confusion'       counts after alias mapping (tolerates renames)
#  - 'confusion_strict' counts strict expected vs. predicted
rows = []
confusion = defaultdict(Counter)        # alias-adjusted
confusion_strict = defaultdict(Counter) # strict

def is_alias_match(expected: str, predicted: str) -> bool:
    """True if predicted equals expected OR equals alias(expected)."""
    return predicted == expected or predicted == INTENT_ALIASES.get(expected, None)

for text, expected in TEST_ITEMS:
    try:
        # Call the Assistant with the test utterance.
        resp = assistant.message(
            workspace_id=WORKSPACE_ID,
            input={"text": text},
        ).get_result()

        # Extract the top intent and its confidence if available.
        intents = resp.get("intents", [])
        top_intent = intents[0]["intent"] if intents else "NONE"
        conf = float(intents[0]["confidence"]) if intents else 0.0

        # strict_ok: requires exact label match
        strict_ok = (top_intent == expected)
        # alias_ok: allows a renamed label via INTENT_ALIASES
        alias_ok  = is_alias_match(expected, top_intent)

        # Record one row per utterance for later tabular reporting.
        rows.append({
            "utterance": text,
            "expected_intent": expected,
            "predicted_intent": top_intent,
            "confidence": round(conf, 3),
            "strict_correct": strict_ok,
            "alias_correct": alias_ok
        })

        # Update confusion counts
        expected_alias = INTENT_ALIASES.get(expected, expected)
        confusion_strict[expected][top_intent] += 1
        confusion[expected_alias][top_intent] += 1

    except Exception as e:
        # If something fails (e.g., network), mark as ERROR so students see it in the table.
        rows.append({
            "utterance": text,
            "expected_intent": expected,
            "predicted_intent": "ERROR",
            "confidence": 0.0,
            "strict_correct": False,
            "alias_correct": False,
            "error": str(e),
        })
        expected_alias = INTENT_ALIASES.get(expected, expected)
        confusion_strict[expected]["ERROR"] += 1
        confusion[expected_alias]["ERROR"] += 1

# Convert list of dicts → DataFrame for easy metrics & printing.
df = pd.DataFrame(rows)

# ---- Metrics -----------------------------------------------------------------
N = len(df)

# Strict accuracy: fraction of examples where predicted_intent == expected_intent
strict_acc = df["strict_correct"].mean() if N else 0.0

# Alias-adjusted accuracy: strict OR predicted equals the alias(expected)
alias_acc  = df["alias_correct"].mean() if N else 0.0

print("\n=== Watson Assistant Classic — Flower Shop Evaluation ===")
print(f"Workspace: {WORKSPACE_ID}")
print(f"Samples:   {N}")
print(f"Strict Accuracy:         {strict_acc:.2%}")
print(f"Alias-Adjusted Accuracy: {alias_acc:.2%}\n")

# Show a compact per-row report to help students debug predictions and labels.
print(
    df[["utterance","expected_intent","predicted_intent","confidence","strict_correct","alias_correct"]]
    .to_string(index=False)
)

# ---- Confusion matrix (alias-adjusted) --------------------------------------
# Rows: expected AFTER alias mapping; Cols: predicted labels observed.
labels_rows = sorted(set(INTENT_ALIASES.get(e, e) for _, e in TEST_ITEMS))
labels_cols = sorted(set(df["predicted_intent"]))

cm = pd.DataFrame(
    [[confusion[r][c] for c in labels_cols] for r in labels_rows],
    index=labels_rows, columns=labels_cols
)

print("\n=== Confusion Matrix (rows=expected after alias, cols=predicted) ===")
print(cm.to_string())

# ---- (Optional) Persist raw results for later grading or class discussion ----
out_csv = "watson_flower_shop_eval.csv"
df.to_csv(out_csv, index=False)
print(f"\nSaved detailed results to {out_csv}")

# ---------------------- Teaching Takeaways ------------------------------------
# • If Strict vs Alias-Adjusted are far apart, your workspace labels differ from the rubric—
#   fix aliases or rename intents to stabilize evaluation.
# • Use the confusion matrix to find systematic mix-ups (e.g., 'hours' vs 'store_location').
# • Extend this cell by adding precision/recall per intent or macro/micro F1 if desired.




=== Intents in workspace ===
['about_you', 'flower_recommendations', 'goodbyes', 'greetings', 'hours_info', 'location_info', 'thank_you']

⚠️  Warning: Some expected labels (after alias mapping) are not in this workspace:
  - expected 'availability' → mapped 'availability'  (NOT FOUND)

=== Watson Assistant Classic — Flower Shop Evaluation ===
Workspace: d1873eca-ba94-4aed-be7b-f8def6f59f5b
Samples:   15
Strict Accuracy:         0.00%
Alias-Adjusted Accuracy: 86.67%

                                    utterance   expected_intent       predicted_intent  confidence  strict_correct  alias_correct
           Which flowers for Valentine's Day? recommend_flowers flower_recommendations       0.957           False           True
                 What flowers do you suggest? recommend_flowers flower_recommendations       1.000           False           True
        Flowers suggestions for my girlfriend recommend_flowers flower_recommendations       1.000           False           True
       

In [13]:
# ====================== OPTIONAL: Per-intent Precision / Recall / F1 ======================
# Uses ALIAS-ADJUSTED ground truth so evaluation tolerates renamed intents.

# 1) Build alias-adjusted ground truth vs predictions
df["expected_alias"] = df["expected_intent"].map(lambda e: INTENT_ALIASES.get(e, e))
y_true = df["expected_alias"].fillna("NONE")
y_pred = df["predicted_intent"].fillna("NONE")

labels = sorted(set(y_true) | set(y_pred))

# 2) Compute per-label counts: TP, FP, FN
per_label = {}
total_tp = total_fp = total_fn = 0
supports = {}

for L in labels:
    tp = int(((y_true == L) & (y_pred == L)).sum())
    fp = int(((y_true != L) & (y_pred == L)).sum())
    fn = int(((y_true == L) & (y_pred != L)).sum())
    support = int((y_true == L).sum())
    supports[L] = support

    # safe divisions
    prec = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    rec  = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1   = (2 * prec * rec / (prec + rec)) if (prec + rec) > 0 else 0.0

    per_label[L] = {
        "precision": round(prec, 3),
        "recall":    round(rec, 3),
        "f1":        round(f1, 3),
        "support":   support,
        "tp": tp, "fp": fp, "fn": fn,
    }

    total_tp += tp
    total_fp += fp
    total_fn += fn

per_label_df = pd.DataFrame.from_dict(per_label, orient="index")[["precision","recall","f1","support","tp","fp","fn"]]
print("\n=== Per-Intent Metrics (alias-adjusted ground truth) ===")
print(per_label_df.to_string())

# 3) Micro / Macro / Weighted averages
# Micro: pool all decisions across labels
micro_prec = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0.0
micro_rec  = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0.0
micro_f1   = (2 * micro_prec * micro_rec / (micro_prec + micro_rec)) if (micro_prec + micro_rec) > 0 else 0.0

# Macro: unweighted mean over labels
macro_prec = per_label_df["precision"].mean() if not per_label_df.empty else 0.0
macro_rec  = per_label_df["recall"].mean()    if not per_label_df.empty else 0.0
macro_f1   = per_label_df["f1"].mean()        if not per_label_df.empty else 0.0

# Weighted: mean weighted by class support
total_support = per_label_df["support"].sum()
if total_support > 0:
    weighted_prec = float((per_label_df["precision"] * per_label_df["support"]).sum() / total_support)
    weighted_rec  = float((per_label_df["recall"]    * per_label_df["support"]).sum() / total_support)
    weighted_f1   = float((per_label_df["f1"]        * per_label_df["support"]).sum() / total_support)
else:
    weighted_prec = weighted_rec = weighted_f1 = 0.0

summary_df = pd.DataFrame(
    [
        ["micro",   round(micro_prec,3),   round(micro_rec,3),   round(micro_f1,3)],
        ["macro",   round(macro_prec,3),   round(macro_rec,3),   round(macro_f1,3)],
        ["weighted",round(weighted_prec,3),round(weighted_rec,3),round(weighted_f1,3)],
    ],
    columns=["average", "precision", "recall", "f1"]
)
print("\n=== Averages (alias-adjusted) ===")
print(summary_df.to_string(index=False))

# ---------------- (Optional) STRICT variant for comparison -------------------
# Uncomment to compute the same metrics using strict labels (no alias tolerance).
# y_true_strict = df["expected_intent"].fillna("NONE")
# y_pred_strict = df["predicted_intent"].fillna("NONE")
# labels_strict = sorted(set(y_true_strict) | set(y_pred_strict))
# 
# def compute_metrics(y_true, y_pred, labels):
#     total_tp = total_fp = total_fn = 0
#     rows = []
#     for L in labels:
#         tp = int(((y_true == L) & (y_pred == L)).sum())
#         fp = int(((y_true != L) & (y_pred == L)).sum())
#         fn = int(((y_true == L) & (y_pred != L)).sum())
#         support = int((y_true == L).sum())
#         prec = tp / (tp + fp) if (tp + fp) > 0 else 0.0
#         rec  = tp / (tp + fn) if (tp + fn) > 0 else 0.0
#         f1   = (2 * prec * rec / (prec + rec)) if (prec + rec) > 0 else 0.0
#         rows.append((L, round(prec,3), round(rec,3), round(f1,3), support))
#         total_tp += tp; total_fp += fp; total_fn += fn
#     per_label = pd.DataFrame(rows, columns=["label","precision","recall","f1","support"]).set_index("label")
#     micro_prec = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0.0
#     micro_rec  = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0.0
#     micro_f1   = (2 * micro_prec * micro_rec / (micro_prec + micro_rec)) if (micro_prec + micro_rec) > 0 else 0.0
#     macro_prec = per_label["precision"].mean() if not per_label.empty else 0.0
#     macro_rec  = per_label["recall"].mean()    if not per_label.empty else 0.0
#     macro_f1   = per_label["f1"].mean()        if not per_label.empty else 0.0
#     total_support = per_label["support"].sum()
#     if total_support > 0:
#         weighted_prec = float((per_label["precision"] * per_label["support"]).sum() / total_support)
#         weighted_rec  = float((per_label["recall"]    * per_label["support"]).sum() / total_support)
#         weighted_f1   = float((per_label["f1"]        * per_label["support"]).sum() / total_support)
#     else:
#         weighted_prec = weighted_rec = weighted_f1 = 0.0
#     avg = pd.DataFrame(
#         [
#             ["micro",   round(micro_prec,3),   round(micro_rec,3),   round(micro_f1,3)],
#             ["macro",   round(macro_prec,3),   round(macro_rec,3),   round(macro_f1,3)],
#             ["weighted",round(weighted_prec,3),round(weighted_rec,3),round(weighted_f1,3)],
#         ],
#         columns=["average", "precision", "recall", "f1"]
#     )
#     return per_label, avg
# 
# strict_per_label, strict_avg = compute_metrics(y_true_strict, y_pred_strict, labels_strict)
# print("\n=== Per-Intent Metrics (STRICT) ===")
# print(strict_per_label.to_string())
# print("\n=== Averages (STRICT) ===")
# print(strict_avg.to_string(index=False))


=== Per-Intent Metrics (alias-adjusted ground truth) ===
                        precision  recall     f1  support  tp  fp  fn
availability                  0.0     0.0  0.000        2   0   0   2
flower_recommendations        1.0     1.0  1.000        5   5   0   0
hours_info                    0.8     1.0  0.889        4   4   1   0
location_info                 0.8     1.0  0.889        4   4   1   0

=== Averages (alias-adjusted) ===
 average  precision  recall    f1
   micro      0.867   0.867 0.867
   macro      0.650   0.750 0.694
weighted      0.760   0.867 0.807
